In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.stats import norm


train_preds = pd.read_csv("TrainingLong/TrainingPredictionsLong_Gaussian.csv")
train_std = pd.read_csv("TrainingLong/TrainingStdLong_Gaussian.csv")

val_preds = pd.read_csv("ValidationLong/ValidationPredictionsLong_Gaussian.csv")
val_std = pd.read_csv("ValidationLong/ValidationStdLong_Gaussian.csv")

test_preds = pd.read_csv("TestingLong/TestingPredictionsLong_Gaussian.csv")
test_std = pd.read_csv("TestingLong/TestingStdLong_Gaussian.csv")

train_actual = pd.read_csv("TrainingLong/TrainingActual_Long.csv")
val_actual = pd.read_csv("ValidationLong/ValidationActual_Long.csv")
test_actual = pd.read_csv("TestingLong/TestingActual_Long.csv")

# Define a function to calculate the encapsulation percentage
def calculate_encapsulation_percentage(confidence=0.7, data_name ="Training", data_setting = "Long"):
    
    prediction_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Predictions{}_Gaussian.csv".format(data_name, data_setting, data_name, data_setting)
    actual_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Actual_{}.csv".format(data_name,data_setting,data_name,data_setting)
    std_dev_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Std{}_Gaussian.csv".format(data_name, data_setting, data_name, data_setting)
    
    prediction_df = pd.read_csv(prediction_df)
    actual_df = pd.read_csv(actual_df)
    std_dev_df = pd.read_csv(std_dev_df)
    
    # Calculate Z-score for the given confidence level
    Z = norm.ppf((1 + confidence) / 2)

    # Ensure numeric conversion to handle non-numeric data gracefully
    actual_df = actual_df.apply(pd.to_numeric, errors='coerce')
    prediction_df = prediction_df.apply(pd.to_numeric, errors='coerce')
    std_dev_df = std_dev_df.apply(pd.to_numeric, errors='coerce')

    # Compute the confidence interval bounds
    lower_bounds = prediction_df - Z * std_dev_df
    upper_bounds = prediction_df + Z * std_dev_df

    # Check if actual values fall within the bounds
    encapsulation = (actual_df >= lower_bounds) & (actual_df <= upper_bounds)

    # Calculate the percentage of encapsulated values
    encapsulation_percentage = encapsulation.sum().sum() / encapsulation.size * 100

    return encapsulation_percentage

# Test the function with the loaded data
#encapsulation_result = calculate_encapsulation_percentage(data_setting = "Short", data_name = "Training", confidence = 0.95)
#encapsulation_result


In [5]:
from ast import literal_eval 

val_100_short = pd.read_csv("/home/jik19004/FilesToRun/BayesianTimeSeries/ValidationShort/hundred_preds_short_Gaussian.csv")
array = np.ones(shape = val_100_short.shape)

for i in range(len(val_100_short)):
    for j in range(val_100_short.shape[1]):
         arr = val_100_short.iloc[i][j] # get the lsit 
         arr = literal_eval(arr) 
         arr = [float(x) for x in arr] # convert each to floats. 
         
         array[i][j] = np.median(arr) # get the median 
         

# Taking account of Laplacian Distribution


In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy.stats import norm, laplace 


# Define a function to calculate the encapsulation percentage
def calculate_encapsulation_percentage(confidence=0.7, data_name ="Training", data_setting = "Long"):
    
    actual_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Actual_{}.csv".format(data_name,data_setting,data_name,data_setting)
    predictions_100_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/hundred_preds_{}_Gaussian.csv".format(data_name, data_setting, data_setting)
    predictions_100_df = pd.read_csv(predictions_100_df)
    actual_df = pd.read_csv(actual_df)
    
    if "Unnamed: 0" in predictions_100_df.columns.values:
        predictions_100_df.drop(["Unnamed: 0"], axis = 1, inplace = True)

    
    
    median_array = np.ones(shape = predictions_100_df.shape)
    mad_array = np.ones(shape = predictions_100_df.shape) # create for mad_array 
    
    confidence = (1 - confidence)
    
    for i in range(len(predictions_100_df)):
        for j in range(predictions_100_df.shape[1]):
            arr = predictions_100_df.iloc[i][j] # get the lsit 
            arr = literal_eval(arr) 
            arr = [float(x) for x in arr] # convert each to floats. 
            
            median_array[i][j] = np.median(arr) # get the median 
            
    encapsulation_raw = 0  
    exceedance_lower = 0 
    exceedance_upper = 0            
    for i in range(len(predictions_100_df)):
        for j in range(predictions_100_df.shape[1]):
            arr = predictions_100_df.iloc[i][j] # get the lsit 
            arr = literal_eval(arr) 
            arr = [float(x) for x in arr] # convert each to floats. 
            
            mad_array[i][j] = np.median(np.abs(arr - median_array[i][j])) # get the mad value.  
            b = mad_array[i][j] / np.log(2)
            
            ci_lower = median_array[i][j] + b * np.log(confidence)
            ci_upper = median_array[i][j] - b * np.log(confidence)
            
            if ci_lower <= actual_df.iloc[i][j] and ci_upper >= actual_df.iloc[i][j]:
                encapsulation_raw +=1 
            elif actual_df.iloc[i][j] < ci_lower:
                exceedance_lower += 1
            else:
                exceedance_upper += 1 
            
        

    # Calculate the percentage of encapsulated values
    encapsulation_percentage = encapsulation_raw / (predictions_100_df.shape[0] * predictions_100_df.shape[1]) * 100
    exceedance_upper = exceedance_upper/(predictions_100_df.shape[0] * predictions_100_df.shape[1]) * 100
    exceedance_lower = exceedance_lower/(predictions_100_df.shape[0] * predictions_100_df.shape[1]) * 100

    return encapsulation_percentage, exceedance_upper, exceedance_lower

# Test the function with the loaded data
"""
encapsulation_result = calculate_encapsulation_percentage(data_setting = "Long", data_name = "Testing", confidence = 0.95)
encapsulation_result """


'\nencapsulation_result = calculate_encapsulation_percentage(data_setting = "Long", data_name = "Testing", confidence = 0.95)\nencapsulation_result '

# Exceedance Frequency Both Upper and Lower 

In [7]:
def calculate_encapsulation_percentage(confidence = 0.7, data_name ="Training", data_setting = "Long", bound_setting = "Upper"):
    prediction_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Predictions{}_Gaussian.csv".format(data_name, data_setting, data_name, data_setting) 
    actual_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Actual_{}.csv".format(data_name,data_setting,data_name,data_setting) 
    std_dev_df = "/home/jik19004/FilesToRun/BayesianTimeSeries/{}{}/{}Std{}_Gaussian.csv".format(data_name, data_setting, data_name, data_setting) 
    
    prediction_df = pd.read_csv(prediction_df) 
    actual_df = pd.read_csv(actual_df) 
    std_dev_df = pd.read_csv(std_dev_df)  

    Z = norm.ppf((1 + confidence) / 2) 


    actual_df = actual_df.apply(pd.to_numeric, errors='coerce') 
    prediction_df = prediction_df.apply(pd.to_numeric, errors='coerce') 
    std_dev_df = std_dev_df.apply(pd.to_numeric, errors='coerce') 


    lower_bounds = prediction_df - Z * std_dev_df 
    upper_bounds = prediction_df + Z * std_dev_df 

    excedence = 0
    if bound_setting is "Upper":
        excedence = actual_df > upper_bounds # strictly greater than upper bound. 
    else:
        excedence = actual_df < lower_bounds 
    
    encapsulation_percentage = excedence.sum().sum() / excedence.size * 100

    return encapsulation_percentage


<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_30616/1853143965.py:22: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if bound_setting is "Upper":


In [10]:
value_upper = calculate_encapsulation_percentage(confidence=0.7, data_name="Testing", data_setting = "Short", bound_setting="Upper")
value_lower = calculate_encapsulation_percentage(confidence=0.7, data_name="Testing", data_setting = "Short", bound_setting="Lower")
print(value_upper)
print(value_lower)
print(100-(value_lower + value_upper))

8.03741886216113
1.9186712485681559
90.04390988927071


In [6]:
no

NameError: name 'no' is not defined